In [1]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
#Limpieza y carga del dataframe

def limpieza_data():

    path = "accepted_2007_to_2018Q4.csv"
    df = pd.read_csv(path)

    # Limpieza de NaNs
    num = 150000
    num_nans = df.isnull().sum()
    columns_menos_nans = num_nans[num_nans < num].index
    df_sin_nans = df[columns_menos_nans]

    df_limpio = df_sin_nans[[
        "id",
        "loan_amnt",
        "funded_amnt",
        "term",
        "int_rate",
        "installment",
        "grade",
        "emp_length",
        "annual_inc",
        "dti",
        "fico_range_high",
        "fico_range_low",
        "revol_util",
        "loan_status",
        "out_prncp",
        "total_rec_prncp",
        "total_rec_int",
        "last_fico_range_high",
        "last_fico_range_low",
        "open_acc",
        "pub_rec",
        "inq_last_6mths",
        "home_ownership",
        "recoveries",
        "num_tl_90g_dpd_24m",
        "purpose",
        "delinq_2yrs"
    ]]

    return df_limpio

df_limpio = limpieza_data()

C:\Users\rabad\AppData\Local\Temp\ipykernel_1492\3507408007.py:6: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [3]:
pd.set_option("display.max_columns", None)
df_limpio

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,emp_length,annual_inc,dti,fico_range_high,fico_range_low,revol_util,loan_status,out_prncp,total_rec_prncp,total_rec_int,last_fico_range_high,last_fico_range_low,open_acc,pub_rec,inq_last_6mths,home_ownership,recoveries,num_tl_90g_dpd_24m,purpose,delinq_2yrs
0,68407277,3600.0,3600.0,36 months,13.99,123.03,C,10+ years,55000.0,5.91,679.0,675.0,29.7,Fully Paid,0.00,3600.00,821.72,564.0,560.0,7.0,0.0,1.0,MORTGAGE,0.0,0.0,debt_consolidation,0.0
1,68355089,24700.0,24700.0,36 months,11.99,820.28,C,10+ years,65000.0,16.06,719.0,715.0,19.2,Fully Paid,0.00,24700.00,979.66,699.0,695.0,22.0,0.0,4.0,MORTGAGE,0.0,0.0,small_business,1.0
2,68341763,20000.0,20000.0,60 months,10.78,432.66,B,10+ years,63000.0,10.78,699.0,695.0,56.2,Fully Paid,0.00,20000.00,2705.92,704.0,700.0,6.0,0.0,0.0,MORTGAGE,0.0,0.0,home_improvement,0.0
3,66310712,35000.0,35000.0,60 months,14.85,829.90,C,10+ years,110000.0,17.06,789.0,785.0,11.6,Current,15897.65,19102.35,12361.66,679.0,675.0,13.0,0.0,0.0,MORTGAGE,0.0,0.0,debt_consolidation,0.0
4,68476807,10400.0,10400.0,60 months,22.45,289.91,F,3 years,104433.0,25.37,699.0,695.0,64.5,Fully Paid,0.00,10400.00,1340.50,704.0,700.0,12.0,0.0,3.0,MORTGAGE,0.0,0.0,major_purchase,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260696,88985880,40000.0,40000.0,60 months,10.49,859.56,B,9 years,227000.0,12.75,709.0,705.0,64.9,Current,23252.59,16747.41,8156.52,724.0,720.0,5.0,0.0,1.0,MORTGAGE,0.0,7.0,debt_consolidation,7.0
2260697,88224441,24000.0,24000.0,60 months,14.49,564.56,C,6 years,110000.0,18.30,664.0,660.0,68.1,Charged Off,0.00,3521.91,3233.49,594.0,590.0,10.0,1.0,0.0,RENT,0.0,0.0,debt_consolidation,0.0
2260698,88215728,14000.0,14000.0,60 months,14.49,329.33,C,10+ years,95000.0,23.36,664.0,660.0,54.0,Current,8456.12,5543.88,4027.96,669.0,665.0,8.0,0.0,1.0,MORTGAGE,0.0,0.0,debt_consolidation,0.0
2260699,Total amount funded in policy code 1: 1465324575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Quitamos las dos ultimas filas del dataframe, ya que no nos sirven
df_limpio = df_limpio.drop(df_limpio.tail(2).index)

# Limpieza de columnas
#Limpieza de las filas con nans en base a term, ya que no sirven
df_limpio = df_limpio.dropna(subset = ["term"])

# Columna term a term_months para dejar solo un int8
df_limpio["term_months"] = pd.to_numeric(df_limpio["term"].str.extract(r"(\d+)")[0], errors = "coerce").astype(np.int8)
df_limpio = df_limpio.drop(columns=["term"])

# Columna emp_length a emp_length_years para eliminar la cadena "years"
df_limpio["emp_length_years"] = df_limpio["emp_length"].str.replace(r"[^0-9+]", "", regex = True)
df_limpio = df_limpio.drop(columns=["emp_length"])

#Optimización de espacio del DataFrame

def optimizar_dataframe(df):
    for columna in df.columns:

        if pd.api.types.is_numeric_dtype(df[columna]):

            if df[columna].dtype == "int64":
                df[columna] = pd.to_numeric(df[columna], downcast = "integer")

            elif df[columna].dtype == "float64":
                df[columna] = pd.to_numeric(df[columna], downcast = "float")  # Puede ser float32 o float16

        elif pd.api.types.is_object_dtype(df[columna]):

            if df[columna].nunique() < 0.5 * len(df[columna]):
                df[columna] = df[columna].astype("category")
    
    return df

df_limpio = optimizar_dataframe(df_limpio)

#emp_lentgh_years daba problemas con los nans, por lo que añadiremos un valor categorico "?"

df_limpio["emp_length_years"] = df_limpio["emp_length_years"].astype("category")
df_limpio["emp_length_years"] = df_limpio["emp_length_years"].cat.add_categories("?")
df_limpio["emp_length_years"] = df_limpio["emp_length_years"].fillna("?")

#Transformamos a numérico las columnas que quedan con nans, para su posterior tratamiento
nan_columns = df_limpio.columns[df_limpio.isna().any()].tolist()
for col in nan_columns:
    df_limpio[col] = pd.to_numeric(df_limpio[col], errors = "coerce")

#Daba problemas el manejo de Nans en estas columnas, por lo que lo sustituiremos por un valor conocido "0.0" para que no de problemas y quede como desconocido

float_nan_columns = df_limpio[nan_columns].select_dtypes(include=["float64", "float32"]).columns.tolist()
df_limpio[float_nan_columns] = df_limpio[float_nan_columns].fillna("0.0")

In [6]:
df_limpio.to_csv("datos_prestamos_limpio.csv", index = False)

In [ ]:
#Dividimos en partes el dataframe para aligerar su posterior carga a la base datos SQL

filas_por_parte = 100000

df_partes = np.array_split(df_limpio, len(df_limpio) // filas_por_parte)
df_partes[1]

In [ ]:
#Subida de datos a MySQL a la database: loan_data, tabla: loans

conexion = mysql.connector.connect(
    host = "localhost",  
    user = "root",  
    password = "pass12345"
)

cursor = conexion.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS loan_data;")
conexion.commit()

cursor.execute("USE loan_data;")

cursor.execute('''
CREATE TABLE IF NOT EXISTS loans (
    id BIGINT PRIMARY KEY,
    loan_amnt FLOAT,
    funded_amnt FLOAT,
    int_rate FLOAT,
    installment FLOAT,
    grade CHAR(1),
    annual_inc FLOAT,
    dti FLOAT,
    fico_range_high INT,
    fico_range_low INT,
    revol_util FLOAT,
    loan_status VARCHAR(20),
    out_prncp FLOAT,
    total_rec_prncp FLOAT,
    total_rec_int FLOAT,
    last_fico_range_high INT,
    last_fico_range_low INT,
    open_acc INT,
    pub_rec INT,
    inq_last_6mths INT,
    home_ownership VARCHAR(20),
    recoveries FLOAT,
    num_tl_90g_dpd_24m INT,
    purpose VARCHAR(50),
    delinq_2yrs INT,
    term_months INT,
    emp_length_years VARCHAR(10)
);
''')

conexion.commit()

cursor.execute("ALTER TABLE loans DISABLE KEYS;")

for parte_index, df_parte in enumerate(df_partes):
    
    for index, row in df_parte.iterrows():

        sql = f"""
        INSERT INTO loans ({", ".join(df_limpio.columns)}) 
        VALUES ({", ".join(["%s"] * df_limpio.shape[1])});
        """

        val = (
            row['id'], row['loan_amnt'], row['funded_amnt'], row['int_rate'], row['installment'], row['grade'], 
            row['annual_inc'], row['dti'], row['fico_range_high'], row['fico_range_low'], row['revol_util'], 
            row['loan_status'], row['out_prncp'], row['total_rec_prncp'], row['total_rec_int'], 
            row['last_fico_range_high'], row['last_fico_range_low'], row['open_acc'], row['pub_rec'], 
            row['inq_last_6mths'], row['home_ownership'], row['recoveries'], row['num_tl_90g_dpd_24m'], 
            row['purpose'], row['delinq_2yrs'], row['term_months'], row['emp_length_years']
        )

        try:
            cursor.execute(sql, val)
            conexion.commit()
            print(f"Fila insertada: {row['id']} de la parte {parte_index + 1}")
        
        except mysql.connector.Error as err:
            #print(f"Error al insertar fila {row['id']}: {err}")
            conexion.rollback()

cursor.execute("ALTER TABLE loans ENABLE KEYS;")

cursor.close()
conexion.close()

In [2]:
#Conexion con SQL para descargar los datos en un datadrame (df)

conexion = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "pass12345",
    database = "loan_data"
)

cursor = conexion.cursor()

query = "SELECT * FROM loans;"
cursor.execute(query)

data = cursor.fetchall()

columnas = [i[0] for i in cursor.description]

df = pd.DataFrame(data, columns = columnas)

print(df.head())

cursor.close()
conexion.close()

      id  loan_amnt  funded_amnt  int_rate  installment grade  annual_inc  \
0  54734    25000.0      25000.0     11.89       829.10     B     85000.0   
1  55716     3500.0       3500.0     24.99       139.15     E     30784.0   
2  55742     7000.0       7000.0     10.71       228.22     B     65000.0   
3  56121     8000.0       8000.0      6.49       245.16     A     65000.0   
4  56705    11000.0      11000.0      9.99       354.89     B     33500.0   

     dti  fico_range_high  fico_range_low  ...  open_acc pub_rec  \
0  19.48              739             735  ...        10       0   
1  16.96              669             665  ...        14       1   
2  14.29              709             705  ...         7       0   
3  10.84              709             705  ...         5       1   
4  18.38              704             700  ...        11       0   

   inq_last_6mths  home_ownership  recoveries  num_tl_90g_dpd_24m  \
0             0.0            RENT         0.0              

In [3]:
#Comprobación de la descarga correcta del dataframe

pd.set_option("display.max_columns", None)
df

,id,loan_amnt,funded_amnt,int_rate,installment,grade,annual_inc,dti,fico_range_high,fico_range_low,revol_util,loan_status,out_prncp,total_rec_prncp,total_rec_int,last_fico_range_high,last_fico_range_low,open_acc,pub_rec,inq_last_6mths,home_ownership,recoveries,num_tl_90g_dpd_24m,purpose,delinq_2yrs,term_months,emp_length_years
0,54734,25000.0,25000.0,11.89,829.10,B,85000.0,19.48,739,735,52.1,Fully Paid,0.00,25000.00,4330.36,834,830,10,0,0.0,RENT,0.0,NaN,debt_consolidation,0,36,1
1,55716,3500.0,3500.0,24.99,139.15,E,30784.0,16.96,669,665,74.0,Fully Paid,0.00,3500.00,1237.26,654,650,14,1,4.0,RENT,0.0,0.0,debt_consolidation,0,36,1
2,55742,7000.0,7000.0,10.71,228.22,B,65000.0,14.29,709,705,76.7,Fully Paid,0.00,7000.00,1215.54,679,675,7,0,0.0,RENT,0.0,NaN,credit_card,0,36,1
3,56121,8000.0,8000.0,6.49,245.16,A,65000.0,10.84,709,705,35.4,Charged Off,0.00,4477.11,668.37,519,515,5,1,0.0,MORTGAGE,0.0,0.0,home_improvement,0,36,10+
4,56705,11000.0,11000.0,9.99,354.89,B,33500.0,18.38,704,700,44.3,Fully Paid,0.00,11000.00,1793.23,699,695,11,0,0.0,MORTGAGE,0.0,0.0,debt_consolidation,0,36,10+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2257914,145645172,23000.0,23000.0,11.31,756.38,B,145000.0,29.62,674,670,93.2,Current,21915.70,1084.30,414.01,684,680,11,0,0.0,MORTGAGE,0.0,0.0,debt_consolidation,0,36,3
2257915,145645185,25000.0,25000.0,15.02,595.02,C,75000.0,32.93,729,725,66.2,Current,24432.30,567.74,591.01,789,785,9,0,0.0,MORTGAGE,0.0,0.0,debt_consolidation,0,60,10+
2257916,145645206,8500.0,8500.0,16.14,299.43,C,80000.0,16.59,759,755,8.2,Current,8127.30,372.70,214.73,744,740,11,0,1.0,MORTGAGE,0.0,0.0,home_improvement,0,36,8
2257917,145647242,9600.0,9600.0,12.98,323.37,B,35704.0,0.84,724,720,11.5,Current,9158.56,441.44,229.54,714,710,5,0,0.0,MORTGAGE,0.0,0.0,home_improvement,0,36,?
